要运行此代码，请按下“运行时”按钮，并在**免费的Tesla T4 Google Colab实例**上点击“运行所有”！
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> 加入我们的Discord社区，如果你需要帮助。并在GitHub上给我们点赞 ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

在自己的计算机上安装Unsloth，请参阅我们GitHub页面上的安装指南 [这里](https://docs.unsloth.ai/get-started/installing-+-updating)。

你将学习如何进行[数据准备](#Data)、如何[训练](#Train)、如何[运行模型](#Inference)和[如何保存模型](#Save)。

访问我们的文档，获取所有[模型上传](https://docs.unsloth.ai/get-started/all-our-models)和[笔记本](https://docs.unsloth.ai/get-started/unsloth-notebooks)的信息。


### 新闻

**阅读我们的[博客文章](https://unsloth.ai/blog/r1-reasoning)以获取训练推理模型的指导。** GRPO 笔记本的灵感来自于 [@shxf0072](https://x.com/shxf0072/status/1886085377146180091)、[@Teknium1](https://x.com/Teknium1/status/1885077369142337550)、[@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb)。

访问我们的文档，获取所有[模型上传](https://docs.unsloth.ai/get-started/all-our-models)和[笔记本](https://docs.unsloth.ai/get-started/unsloth-notebooks)的信息。


### 安装

发生错误: 下载数据集失败，状态码: 404


In [ ]:
%%capture
# 在Colab中跳过重启消息
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# 如果你在本地运行这个笔记本，需要安装 `diffusers` too
# !pip install diffusers
# 暂时安装一个特定的 TRL nightly 版本
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

使用 `PatchFastRL` 在所有函数之前修补 GRPO 和其他 RL 算法！

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

加载 `Qwen 2.5 3B Instruct` 模型并设置参数

In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024  # 可以增加以支持更长的推理跟踪
lora_rank = 64  # 更大的 rank 意味着更智能，但速度更慢

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False 用于 LoRA 16bit
    fast_inference=True,  # 启用 vLLM 快速推理
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.5,  # 如果内存不足，请减小此值
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,  # 选择任何大于 0 的数！建议 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # 如果内存不足，请移除 QKVO
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",  # 启用长上下文微调
    random_state=3407,
)

### 数据准备
<a name="Data"></a>

我们直接使用了 [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) 的数据准备方法和所有奖励函数。你可以自由创建自己的数据准备方法！

In [ ]:
import re
from datasets import load_dataset, Dataset

# 加载并准备数据集
SYSTEM_PROMPT = """
响应格式如下：
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    """从文本中提取 XML 格式的答案"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    """从文本中提取带有哈希的答案"""
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    """获取 GSM8K 数据集的题目"""
    data = load_dataset('openai/gsm8k', 'main')[split]  # type: ignore
    data = data.map(lambda x: {  # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })  # type: ignore
    return data  # type: ignore

dataset = get_gsm8k_questions()

# 奖励函数
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """正确性奖励函数"""
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    """整数奖励函数"""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """严格格式奖励函数"""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """软格式奖励函数"""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    """计算 XML 格式的得分"""
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """XML计数奖励函数"""
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
### 训练模型

现在设置 GRPO 训练器和所有配置！

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm=True,  # 使用 vLLM 进行快速推理！
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),  # 是否支持 bfloat16
    fp16=not is_bfloat16_supported(),  # 是否使用 fp16
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # 增加到 4 以获得更平滑的训练
    num_generations=8,  # 如果内存不足，请减少
    max_prompt_length=256,
    max_completion_length=200,
    # num_train_epochs=1,  # 设置为 1 进行完整的训练
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # 可以使用 Weights & Biases
    output_dir="outputs",
)

现在，让我们运行训练器！如果你向上滚动，你会看到一个奖励表。目标是看到 `reward` 列的值增加！

你可能需要等待 150 到 200 个步骤才能看到任何动作。前 100 个步骤可能会得到 0 奖励，请耐心等待！

| 步骤 | 训练损失 | 奖励    | 奖励标准差 | 完成长度 | KL       |
|------|----------|---------|------------|----------|----------|
| 1    | 0.000000 | 0.125000 | 0.000000   | 200.000000 | 0.000000 |
| 2    | 0.000000 | 0.072375 | 0.248112   | 200.000000 | 0.000000 |
| 3    | 0.000000 | -0.079000 | 0.163776   | 182.500000 | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args=training_args,
    train_dataset=dataset,
)
trainer.train()

<a name="Inference"></a>
### 推理
现在，让我们尝试我们刚刚训练的模型！首先，让我们先尝试没有经过 GRPO 训练的模型：

In [ ]:
text = tokenizer.apply_chat_template([
    {"role": "user", "content": "How many r's are in strawberry?"},
], tokenize=False, add_generation_prompt=True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
)
output = model.fast_generate(
    [text],
    sampling_params=sampling_params,
    lora_request=None,
)[0].outputs[0].text

output

现在，让我们使用刚刚用 GRPO 训练的 LoRA 进行测试。首先，我们需要保存 LoRA：

In [ ]:
model.save_lora("grpo_saved_lora")

现在，我们加载 LoRA 并进行测试：

In [ ]:
text = tokenizer.apply_chat_template([
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "How many r's are in strawberry?"},
], tokenize=False, add_generation_prompt=True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
)
output = model.fast_generate(
    text,
    sampling_params=sampling_params,
    lora_request=model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

我们的推理模型性能有了显著提升！虽然它并不总是正确，因为我们只进行了大约一个小时的训练，但如果我们延长训练时间和增加序列长度，模型性能会更好。

<a name="Save"></a>
### 保存模型

#### 保存到 float16 格式以供 VLLM 使用

我们还支持直接保存为 `float16` 格式。选择 `merged_16bit` 以使用 float16，或者 `merged_4bit` 以使用 int4。我们还允许 `lora` 适配器作为回退。使用 `push_to_hub_merged` 可以将模型上传到你的 Hugging Face 账户！你可以在 https://huggingface.co/settings/tokens 获取你的个人令牌。

In [ ]:
# 合并到 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method="merged_16bit", token="")

# 合并到 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method="merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method="merged_4bit", token="")

# 只保存 LoRA 适配器
if False: model.save_pretrained_merged("model", tokenizer, save_method="lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method="lora", token="")

### GGUF / llama.cpp 转换
我们现在支持将模型直接保存为 `GGUF` / `llama.cpp` 格式！我们克隆了 `llama.cpp`，默认保存到 `q8_0`。我们支持所有方法，如 `q4_k_m`。使用 `save_pretrained_gguf` 进行本地保存，使用 `push_to_hub_gguf` 上传到 Hugging Face。

支持的量化方法（完整列表请参阅我们的 [Wiki 页面](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)）：
* `q8_0` - 快速转换。资源消耗较高，但通常可以接受。
* `q4_k_m` - 推荐。使用 Q6_K 对注意力权重（`attention.wv`）和前馈网络权重（`feed_forward.w2`）的一半进行量化，其余部分使用 Q4_K。
* `q5_k_m` - 推荐。使用 Q6_K 对注意力权重（`attention.wv`）和前馈网络权重（`feed_forward.w2`）的一半进行量化，其余部分使用 Q5_K。

[**NEW**] 要微调并自动导出到 Ollama，可以尝试我们的 [Ollama 笔记本](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# 保存为 8bit Q8_0 格式
if False: model.save_pretrained_gguf("model", tokenizer,)
# 记得去 https://huggingface.co/settings/tokens 获取你的令牌！
# 并将 hf 更改为你的用户名！
if False: model.push_to_hub_gguf("hf/model", tokenizer, token="")

# 保存为 16bit GGUF 格式
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method="f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method="f16", token="")

# 保存为 q4_k_m GGUF 格式
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method="q4_k_m", token="")

# 保存为多个 GGUF 选项 - 如果你想保存多个选项，这样会更快！
if False:
    model.push_to_hub_gguf(
        "hf/model",  # 将 hf 更改为你的用户名！
        tokenizer,
        quantization_method=["q4_k_m", "q8_0", "q5_k_m",],
        token="",
    )

现在，你可以使用 `model-unsloth.gguf` 文件或 `model-unsloth-Q4_K_M.gguf` 文件在 `llama.cpp` 或基于 Web 的界面如 Jan 或 Open WebUI 中。Jan 可以在 [这里](https://github.com/janhq/jan) 安装，Open WebUI 可以在 [这里](https://github.com/open-webui/open-webui) 安装。

到这里，我们已经完成了所有步骤！如果你有任何关于 Unsloth 的问题，我们有一个 [Discord](https://discord.gg/unsloth) 频道可以提供帮助！如果你发现任何错误或想了解最新的 LLM 动态，或者需要帮助、加入项目等，欢迎加入我们的 Discord！

其他链接：
1. Llama 3.2 对话式笔记本。[免费 Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. 将微调保存到 Ollama。[免费笔记本](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 视觉微调 - 放射学用例。[免费 Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. 查看我们的文档，获取关于 DPO、ORPO、继续预训练、对话式微调等的更多信息！[文档链接](https://docs.unsloth.ai/get-started/unsloth-notebooks)

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  加入 Discord 获取帮助 + ⭐️ <i>在 <a href="https://github.com/unslothai/unsloth">Github</a> 上点赞我们</i> ⭐️
</div>
